In [274]:
input_file = "input_files/day_22.txt"

with open(input_file) as lines:
    data = lines.read().splitlines()

In [232]:
data = [
    '1,0,1~1,2,1',
    '0,0,2~2,0,2',
    '0,2,3~2,2,3',
    '0,0,4~0,2,4',
    '2,0,5~2,2,5',
    '0,1,6~2,1,6',
    '1,1,8~1,1,9',
]

In [275]:
from typing import NamedTuple

class Brick(NamedTuple):
    x1: int
    y1: int
    z1: int
    x2: int
    y2: int
    z2: int
    
    def overlaps_x_y(self, other):
        '''assumes bricks will always be orthogonal to axis -- no diagonals allowed!'''
        #return max(self.start, other.start) < min(self.stop,other.stop)
        self_start_x, self_stop_x = sorted((self.x1, self.x2))
        self_start_y, self_stop_y = sorted((self.y1, self.y2))
        
        other_start_x, other_stop_x = sorted((other.x1, other.x2))
        other_start_y, other_stop_y = sorted((other.y1, other.y2))

        return max(self_start_x, other_start_x) <= min(self_stop_x, other_stop_x) and \
            max(self_start_y, other_start_y) <= min(self_stop_y,other_stop_y)
        
    def supports(self, other):
        return self.overlaps_x_y(other) and min(other.z1, other.z2) - max(self.z1, self.z2)  == 1
    
    def sink(self, z):
        z_size = abs(self.z1 - self.z2)
        return Brick(self.x1, self.y1, z, self.x2, self.y2, z + z_size)

    def max_z(self):
        return max(self.z1, self.z2)

    def min_z(self):
        return min(self.z1, self.z2)

def parse_input(lines):
    bricks = []
    for line in lines:
        b = Brick(*(int(component) for end_point in line.split('~') for component in end_point.split(',')))
        bricks.append(b)
    return bricks
        
        
bricks = parse_input(data)
bricks.sort(key = lambda b: (min(b.z1, b.z2), max(b.z1, b.z2)))


## Part One

In [279]:
from collections import defaultdict

def arrange(bricks):
    supports = defaultdict(set)
    supported_by = defaultdict(set)
    
    max_index = defaultdict(set)
    max_index[bricks[0].max_z()].add(bricks[0])
    
    ordered = [bricks[0]]
    
    for b in bricks[1:]:
        levels = sorted(max_index.keys(), reverse=True)
        first_support_level = next((
            level for level in levels 
            if level <= b.min_z() and any(support.overlaps_x_y(b) for support in max_index[level])
        ), None)
        if first_support_level is None:
            '''bottom'''
            base = b.sink(1)
            ordered.append(base)
            max_index[base.max_z()].add(base)

        else:
            sunk = b.sink(first_support_level + 1)
            for support in max_index[first_support_level]:
                if support.supports(sunk):                    
                    supports[support].add(sunk)
                    supported_by[sunk].add(support)
                    
            ordered.append(sunk)
            max_index[sunk.max_z()].add(sunk)

    return ordered, supports, supported_by

ordered, supports, supported_by = arrange(bricks)

count = 0

for i, b in enumerate(ordered):
    # set of bricks that b supports
    it_supports = supports[b]
    if all(len(supported_by[supported]) > 1 for supported in it_supports):
        count += 1

print("Part One: ", count)

Part One:  490


In [281]:
def destroy(b):
    destroyed = set() 
    to_destroy = [b]
    while len(to_destroy):
        current = to_destroy.pop()
        destroyed.add(current)
        childred = set(s for s in supports[current] if len(supported_by[s] - destroyed) == 0)
        destroyed = destroyed.union(childred)
        to_destroy.extend(childred)
    return destroyed

count = 0

for b in ordered:     
    count += len(destroy(b)) - 1
count   


96356